## Import libraries

In [1]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from matplotlib.collections import LineCollection
from matplotlib.colors import LogNorm
import matplotlib.colors as colors
from scipy import interpolate
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from rich.progress import track #Need to install rich library

%load_ext rich


## Define functions and parameters

In [2]:
#All parameters are defined in units of pixels and frames

#Conversion factors to International System units (cm and s)
lWalls = 2745 #Length of the tank in pixels. It measures 100 cm.
fps = 50 #Frames per second of the recording. 

#Friction-propulsion force
v0_fricProp_lin = 6
tau_fricProp_lin = 80
def f_fricProp_lin(v, v0_fricProp_lin, tau_fricProp_lin): return (v0_fricProp_lin-v)/tau_fricProp_lin

#Noise constants
sigma_v =  0.5
sigma_phi = 0.2

#Social force: attraction-repulsion
k_att = 0.002
k_rep = 0.005
alpha_rep = 1
d_nn = 160
def f_soc_atracRep(rij, modRij, d_nn, k_att, k_rep, alpha_rep):
    return np.where(modRij > d_nn, k_att*(modRij - d_nn), -k_rep*(d_nn - modRij)**alpha_rep) * rij / modRij
f_soc_atracRep_label = r'$k (r_{ij} - d_{nn})^\alpha \hat{r}_{ij}, \alpha_{att} = 1, \alpha_{rep} = %.2g,\\k_{att} = %.2g,  k_{rep} = %.2g, d_{nn} = %2.g$' %(alpha_rep, k_att, k_rep, d_nn)

#Social force: alignment
mu = 0.03
def f_soc_alig(DeltaV, mu):
    return mu*DeltaV
f_soc_alig_label = r'$\mu (\vec{v}_j - \vec{v}_i), \mu = %.2g$' %(mu)

#Social force: explicit anti-alignment model
#We interpolate the experimental alignment force map
Hx, Hy, H, Hx_arrows, Hy_arrows, x_r, y_r = np.load('alignmentMap_nn_N=39.npy', allow_pickle = True)
X, Y = np.meshgrid((x_r[1:] + x_r[:-1])/2, (y_r[1:] + y_r[:-1])/2, indexing = 'ij')
Hx_filter = gaussian_filter(Hx, sigma = 2)
Hy_filter = gaussian_filter(Hy, sigma = 2)
H_filter = np.linalg.norm([Hx_filter, Hy_filter], axis = 0)
fAlign_x = interpolate.RectBivariateSpline( (x_r[1:] + x_r[:-1])/2, (y_r[1:] + y_r[:-1])/2, Hx_filter)
fAlign_y = interpolate.RectBivariateSpline( (x_r[1:] + x_r[:-1])/2, (y_r[1:] + y_r[:-1])/2, Hy_filter)
mu_aligExp = 1
def f_soc_aligExp_i(DeltaV_i, mu_aligExp):
        return mu_aligExp * np.array([fAlign_x(*DeltaV_i).squeeze(1), fAlign_y(*DeltaV_i).squeeze(1) ]) 
f_soc_aligExp_label = 'Experimental alignment for i'

#Social force calculation
def R_vec(theta): #Rotation matrix to change coordinates
    c, s = np.cos(theta), np.sin(theta)
    R_vec = np.array([[c, -s], [s, c]])
    if type(theta) == np.ndarray:
        return R_vec.transpose((*range(2, 2+theta.ndim), 0 , 1))
    else:
        return R_vec
def f_soc(pos_t, vel_t, thetaI, mu, mu_aligExp, d_nn, k_att, k_rep, alpha_rep, isVoronoiNeighbours, isVoronoiWeights, isExperimentalAlignment, isSocialSwitching_lowerV, i_SocialOn):
    #thetaI: The orientation of the direction of motion of individual i.
    #isVoronoiNeighbours: indicates whether Voronoi neighbours or the nearest neighbour is used.
    #isVoronoiWeights: indicates whether weights based on the metric distance are used for Voronoi neighbours.
    #isExperimentalAlignment: indicates whether alignment is used for the explicit anti-alignment model or the standard model.
    #isSocialSwitching_lowerV: indicates whether social forces are switched for neighbours that move at slower speeds than the individual.
    #i_SocialOn: indicates individuals that are socially active.

    if nParticles > 1:
        rij = pos_t[:,None,:] - pos_t[:,:,None]
        modRij = np.linalg.norm(rij, axis = 0)
        vij = vel_t[:,None,:] - vel_t[:,:,None]

        if isSocialSwitching_lowerV:
            modV = np.linalg.norm(vel_t, axis = 0)
            DeltaModV = modV[None,:] - modV[:,None] 

        if isExperimentalAlignment: vij_i =  np.einsum('imn, nij -> mij', R_vec(-thetaI), vij) #vij components in the direction of motion of i (y component is along the direction of motion of i).

        if isVoronoiNeighbours:
            if isExperimentalAlignment:
                fSoc_alig_i = np.zeros((2,nParticles))
            else:
                fSoc_alig = np.zeros((2,nParticles)) 
            fSoc_atracRep = np.zeros((2,nParticles)) 

            indptr_neigh, neighbours = Delaunay(pos_t.T).vertex_neighbor_vertices #Voronoi neighbours
            wij = None #weights

            for i in np.arange(nParticles)[i_SocialOn]:
                j = neighbours[indptr_neigh[i]:indptr_neigh[i+1]]
                if isSocialSwitching_lowerV: j = j[DeltaModV[i, j] > 0]
                if len(j) != 0:
                    if isVoronoiWeights:
                        wij = 1/modRij[i,j]

                    if isExperimentalAlignment:
                        fSoc_alig_i[:,i] = np.average( np.array([f_soc_aligExp_i(vij_i[:,i,k], mu_aligExp)[:,0] for k in j]), weights=wij, axis = 0 )
                    else:
                        fSoc_alig[:,i] = np.average(f_soc_alig(vij[:,i, j], mu), weights=wij, axis = 1 )
                                            
                    fSoc_atracRep[:,i] = np.average(f_soc_atracRep(rij[:,i,j], modRij[None,i,j], d_nn, k_att, k_rep, alpha_rep),weights=wij, axis = 1 )
                else:
                    if isSocialSwitching_lowerV: i_SocialOn[i] = False

            if isExperimentalAlignment: fSoc_alig = np.einsum('imn, ni -> mi', R_vec(thetaI), fSoc_alig_i) #We need to go back to cartesian coordinates
            
        else:
            nn_i = np.argpartition(modRij, 1, axis = -1)[...,1] #Nearest neighbours

            fSoc_atracRep = np.zeros((2,nParticles)) 

            if isSocialSwitching_lowerV:
                i_SocialOn = np.logical_and(i_SocialOn, DeltaModV[np.arange(nParticles), nn_i] > 0)

            if isExperimentalAlignment:
                fSoc_alig_i = np.zeros((2,nParticles))
                fSoc_alig_i[:,i_SocialOn]  = np.array([ f_soc_aligExp_i(vij_i[:,i, nn_i[i]], mu_aligExp)[:,0] for i in np.arange(nParticles)]).T[:,i_SocialOn]
                fSoc_alig = np.einsum('imn, ni -> mi', R_vec(thetaI), fSoc_alig_i)

            else:
                fSoc_alig = np.zeros((2,nParticles))
                fSoc_alig[:,i_SocialOn]  = f_soc_alig(vij[:,np.arange(nParticles), nn_i], mu)[:,i_SocialOn]
        
            fSoc_atracRep[:,i_SocialOn] =  f_soc_atracRep(rij[:,np.arange(nParticles),nn_i], modRij[None,np.arange(nParticles),nn_i], d_nn, k_att, k_rep, alpha_rep)[:,i_SocialOn]

    return fSoc_atracRep, fSoc_alig, i_SocialOn


In [8]:
#Parameters in International System units (cm and s)

print('k_rep: %.3g' %(k_rep *fps**2))
print('k_att: %.2g' %(k_att *fps**2))
print('d_nn: %.2g' %(d_nn / lWalls*100))
print('mu: %.2g' %(mu * fps))
print('v0: %.2g' %(v0_fricProp_lin/lWalls*100 * fps))
print('tau %.2g' %( tau_fricProp_lin/fps ))
print('sigma_v %.2g' %(sigma_v / lWalls * 100 * fps**(3/2)))
print('sigma_phi: %.2g' %(sigma_phi / lWalls * 100 * fps**(3/2)))

k_rep: 12.5
k_att: 5
d_nn: 5.8
mu: 1.5
v0: 11
tau 1.6
sigma_v 6.4
sigma_phi: 2.6


## Functions for plots

In [3]:
def latexFont(size= 15, labelsize=18, titlesize=20, ticklabelssize=15, legendsize = 18):
    #To use latex font
    plt.rcParams.update({
        "text.usetex": True})

    plt.rcParams["text.latex.preamble"].join([
        r"\usepackage{underscore}"
    ])

    plt.rcParams["font.family"] = 'STIXGeneral'

    plt.rc('font', size=size)          # controls default text sizes
    plt.rc('axes', titlesize=titlesize)     # fontsize of the axes title
    plt.rc('axes', labelsize=labelsize)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=ticklabelssize)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=ticklabelssize)    # fontsize of the tick labels
    plt.rc('legend', fontsize=legendsize)    # legend fontsize
    plt.rc('figure', titlesize=titlesize)  # fontsize of the figure title

def strSave(string, N = 120):
    #To name files without weird characters.
    string = string.replace('textrm', '')
    string = string.replace('$', '')
    string = string.replace('\\', '')
    string = string.replace('=', 'eq')
    string = string.replace('__', '_')
    string = string.replace('/', '-')
    string = string.replace('<', 's')
    string = string.replace('>', 'g')
    string = string.replace(':', '')
    string = string.replace('\n', '')
    string = string.replace('|', '')
    string = string.replace('  ', ' ')
    string = string.replace(' ', '_')
    string = string.replace('.', 'p')
    string = string.replace('(', '')
    string = string.replace(')', '')
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace('{', '-')
    string = string.replace('}', '')
    string = string.replace('?', '')
    string = string.replace('!', '')
    string = string.replace('^', '-')
    string = string.replace('*', 'star')
    string = string.replace(',', 'c')
    string = string.replace('left', 'l')
    string = string.replace('right', 'r')
    string = string.replace('frac', 'f')
    string = string.replace('~', '')
    if len(string) >= N:
        n = int(N/2)
        string = "%s%s" %(string[:n], string[-n:])
    return string

def plotCounts2D(x, y, Bins_x, Bins_y, xLabel='', yLabel='', zLabel_extra = '', title='', folder='.', isDensity = False, isLog_z=False, isLog_x = False, isLog_y = False, ax = None, output='png', tagName = ''):
    H, xedges, yedges = np.histogram2d(x, y, bins = [Bins_x, Bins_y], density = isDensity)
    H = np.ma.masked_values(H, 0)

    if ax == None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()

    if isLog_z==False:
        im = ax.pcolormesh(Bins_x, Bins_y, H.T, cmap = 'viridis_r', zorder = -1,rasterized=True)
    else:
        im = plt.pcolormesh(Bins_x, Bins_y, H.T,  norm=LogNorm(vmin=np.nanmin(H), vmax=np.nanmax(H)), cmap = 'viridis_r', zorder = -1, rasterized=True)

    ax.set_xlim([Bins_x[0], Bins_x[-1]])
    ax.set_ylim([Bins_y[0], Bins_y[-1]])

    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel)
    if isLog_x==True: ax.set_xscale('log')
    if isLog_y==True: ax.set_yscale('log')
    ax.set_title(title)
    fig.colorbar(im, label= 'PDF' + zLabel_extra if isDensity else 'Counts' + zLabel_extra )
    ax.set_aspect(1./ax.get_data_ratio())
    if isLog_z==False:
        fig.savefig("%s/%s.%s" %(folder, strSave('%s%s_%s_%s_%s%s' %('PDF' if isDensity else 'Counts' , zLabel_extra, xLabel, yLabel, title, tagName), ), output ), dpi = 300,  bbox_inches='tight') 
    else:
        fig.savefig("%s/%s.%s" %(folder, strSave('%s_%s%s%s_%s%s_%s%s_log' %('PDF' if isDensity else 'Counts', zLabel_extra, xLabel, '_log' if isLog_x else '', yLabel, '_log' if isLog_y else '', title, tagName) ), output ), dpi = 300,  bbox_inches='tight') 
    plt.close(fig = fig)

def subset_cmap(cmap = plt.cm.get_cmap('Greens_r'), minVal=0.0, maxVal=1.0, nColors=256):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minVal, b=maxVal),
        cmap(np.linspace(minVal, maxVal, 256)), N = nColors)
    return new_cmap

def plotAverage2D_VecField(x, y, zx, zy,  Bins_x, Bins_y, BinsArrow_x, BinsArrow_y, binCenterArrow_x, binCenterArrow_y, arrowCutoffPerc = 0.03, scaleArrow = None, isArrowsNorm = False, xLabel = '', yLabel = '', zLabel = '', zPerc = 5, zLim = [None, None], title = '', folder='.', isLog_x = False, isLog_y = False, isLog_z = False, colorArrow = 'k', cmap = 'YlGn', cmap_top = 0.8, ax = None, tagName='', output='png'):

    Hx, xedges, yedges = np.histogram2d(x, y, bins = [Bins_x, Bins_y], weights = zx)
    Hy, xedges, yedges = np.histogram2d(x, y, bins = [Bins_x, Bins_y], weights = zy)
    H_counts, xedges, yedges = np.histogram2d(x, y, bins = [Bins_x, Bins_y]) 
    H_counts = np.ma.masked_values(H_counts, 0)
    Hx = Hx/H_counts
    Hy = Hy/H_counts
    H = np.linalg.norm((Hx, Hy), axis = 0)

    if zLim == [None, None]:          
        zLim = [np.nanpercentile(H, zPerc), np.nanpercentile(H, 100-zPerc)]
    
    if cmap_top != 1: cmap = subset_cmap( plt.cm.get_cmap(cmap), 0, cmap_top)

    
    if ax == None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()

    if isLog_z == False:
        im = ax.pcolormesh(Bins_x, Bins_y, H.T, vmin=zLim[0], vmax = zLim[-1], cmap=cmap, zorder = -1, rasterized=True)
    else:
        im = ax.pcolormesh(Bins_x, Bins_y, H.T, cmap=cmap, norm = LogNorm(vmin=zLim[0], vmax=zLim[-1]), zorder = -2, rasterized=True)
    fig.colorbar(im, label=zLabel)

    #Arrows
    X, Y =np.meshgrid(binCenterArrow_x, binCenterArrow_y, indexing='ij')
    Hx_arrows, xedges, yedges = np.histogram2d(x, y, bins = [BinsArrow_x, BinsArrow_y], weights = zx)
    Hy_arrows, xedges, yedges = np.histogram2d(x, y, bins = [BinsArrow_x, BinsArrow_y], weights = zy)
    H_counts_arrows, xedges, yedges = np.histogram2d(x, y, bins = [BinsArrow_x, BinsArrow_y]) 
    H_counts_arrows = np.ma.masked_less_equal(H_counts_arrows, arrowCutoffPerc * np.sum(H_counts_arrows) / H_counts_arrows.shape[0] / H_counts_arrows.shape[1])
    Hx_arrows = Hx_arrows/H_counts_arrows
    Hy_arrows = Hy_arrows/H_counts_arrows

    if isArrowsNorm:
        H_arrows_mod = np.linalg.norm([Hx_arrows, Hy_arrows], axis = 0)
        Hx_arrows = Hx_arrows/H_arrows_mod
        Hy_arrows = Hy_arrows/H_arrows_mod

    ax.quiver(X, Y, Hx_arrows, Hy_arrows, scale = scaleArrow, color=colorArrow, zorder = -1)
    
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel)
    if isLog_x==True: ax.set_xscale('log')
    if isLog_y==True: ax.set_yscale('log')
    ax.set_title(title)
    ax.set_xlim([Bins_x[0], Bins_x[-1]])
    ax.set_ylim([Bins_y[0], Bins_y[-1]])
    ax.set_aspect(1./ax.get_data_ratio())
    fig.savefig("%s/%s.%s" %(folder, strSave('%s_%s%s_%s%s_%s%s' %(zLabel, xLabel, '_log' if isLog_x else '', yLabel, '_log' if isLog_y else '', title, '_' + tagName if tagName != '' else '') ), output ), dpi = 300,  bbox_inches='tight') 
    plt.close(fig = fig)

## Simulations

In [8]:
folder = 'sim' #Need to create this folder!!

#Select simulation type --------------------------
isDrawn = False
isVoronoiNeighbours = True
isVoronoiWeights = True
isExperimentalAlignment = False
isSocialSwitching_F_SocialOff = False
isSocialSwitching_lowerV = False

#isDrawn: Whether to save every frame of the simulation as a picture in the folder 'folder'.
#isVoronoiNeighbours: Whether to use Voronoi neighbours or the nearest neighbour.
#isVoronoiWeights: Whether weights based on the metric distance are used for Voronoi neighbours.
#isExperimentalAlignment: Whether alignment is used for the explicit anti-alignment model or the standard model.
#isSocialSwitching_F_SocialOff: Whether social forces are switched off for individuals randomly and instead they feel a persistent random force
#isSocialSwitching_lowerV: Whether social forces are switched off for neighbours that move at slower speeds than the individual.
#-------------------------------------------------------

nParticles = 39 #Number of individuals
tTransient = 2000 #Transient time to discard in the simulations
tMax = 10000 #150000 #Last frame to integrate

dt = 1 #Frame rate for numerical integration

#Parameters for the persistent random force
pSocialOff = 1/20 #Probability rate for an individual to switch off the social interactions
tauSocialOff = 5 #Persistent time to have switched off the social interactions
A_FSocialOff = 0.3 #Amplitude of the persistent random force

rng = np.random.default_rng(1) #Seed for the random number generator 
nIter = int(np.ceil(tMax / dt)) #Number of iterations to perform the integration


#Tag name
tag = 'sim%s%s%s%s%s%s' %(
    '_Vor' if isVoronoiNeighbours else '_Nn',
    '_Wij' if isVoronoiNeighbours and isVoronoiWeights else '',
    '_aligExp' if isExperimentalAlignment else '',
    '_FSocOff_%.1g' %A_FSocialOff if isSocialSwitching_F_SocialOff else '',
    '_lowV' if isSocialSwitching_lowerV else '',
    '_nTime_%d' %(tMax-tTransient))

latexFont(size= 15, labelsize=18, titlesize=20, ticklabelssize=15, legendsize = 18)


#Parameters to draw the simulation  
DtSave = 1 #How often to draw the simulation.
scaleVectors_v = 0.09*lWalls/100
scaleVectors_F = 0.003*lWalls/100
modAcc_lim = 2
widthVectors = 0.0015
alphaVectors = 0.8
color_antisocial = 'k'


lGroup =2*600 #Lenght of the group

#Initial values of the simulation
pos_0 = rng.standard_normal((2,nParticles))*lGroup/2 #Position
vel_0 = rng.uniform(3,8, (2, nParticles)) #Velocity
z_0 = np.array([*pos_0, *vel_0])

# z[0] = x, z[1] = y, z[2] = vx, z[3] = vy
z = np.empty((4,nIter,nParticles))
z[:,0] = z_0
tSave = 0

F_Soc_alig = np.zeros((2,nIter-1,nParticles))
F_Soc_atracRep = np.zeros((2,nIter-1,nParticles))
F_fricProp =  np.empty((2,nIter-1,nParticles))
F_noise = np.empty((2,nIter-1,nParticles))
F_SocialOff = np.zeros((2,nIter-1,nParticles))

i_SocialOn = np.full(nParticles, True)
if isSocialSwitching_F_SocialOff:
    tFinishSocialOff = np.zeros(nParticles)
    pSocialOff_k = pSocialOff * dt

#Integrate the simulation
for k in track(range(1, nIter) ):
    t = k*dt

    v = np.linalg.norm(z[2:,k-1,:], axis = 0)
    uv_vel = z[2:,k-1,:] / v[None,:] #Normalized vector in the direction of motion
    uv_phi = np.array([-uv_vel[1], uv_vel[0]]) #Normalized vector in the direction perpendicular to the direction of motion.
    thetaI = np.arctan2(-z[2,k-1,:], z[3,k-1,:]) #Orientation of individual i.

    if isSocialSwitching_F_SocialOff:
        i_SocialOn = t>=tFinishSocialOff
        if k>=2: F_SocialOff[:,k-1, ~i_SocialOn] = F_SocialOff[:,k-2, ~i_SocialOn]

        newI_SocialOff = np.logical_and(i_SocialOn, rng.random(nParticles) <= pSocialOff_k)
        i_SocialOn[newI_SocialOff] = False
        tFinishSocialOff[newI_SocialOff] = t + tauSocialOff
        theta_interactionsOff  = rng.random(nParticles) * 2*np.pi
        
        F_SocialOff[:,k-1,newI_SocialOff] = A_FSocialOff * np.array([np.cos(theta_interactionsOff[newI_SocialOff]), np.sin(theta_interactionsOff[newI_SocialOff])])        
    elif isSocialSwitching_lowerV: i_SocialOn = np.full(nParticles, True)


    if k>=1: 
        F_Soc_atracRep[:,k-1], F_Soc_alig[:,k-1], i_SocialOn = f_soc(z[:2,k-1,:], z[2:,k-1,:], thetaI, mu, mu_aligExp, d_nn, k_att, k_rep, alpha_rep, isVoronoiNeighbours, isVoronoiWeights, isExperimentalAlignment, isSocialSwitching_lowerV, i_SocialOn)

    F_fricProp[:,k-1] = f_fricProp_lin(v, v0_fricProp_lin, tau_fricProp_lin)*uv_vel

    F_det = F_fricProp[:,k-1] + F_Soc_alig[:,k-1] + F_Soc_atracRep[:,k-1] + F_SocialOff[:,k-1]

    F_noise[:,k-1] = sigma_v * rng.standard_normal(nParticles) * uv_vel + sigma_phi *  rng.standard_normal(nParticles) * uv_phi

    z[:,k] = np.array([ *(z[:2,k-1,:] + dt * z[2:,k-1,:]),
                        *(z[2:,k-1,:] + dt * F_det + np.sqrt(dt) * F_noise[:,k-1])])

    #Draw simulation
    if isDrawn == True and t >= tTransient and t % DtSave == 0:                

        posDraw = z[:2,k-1,:]/lWalls*100

        if isVoronoiNeighbours:
            if nParticles > 1:
                indptr_neigh, neighbours = Delaunay(z[:2,k-1,:].T).vertex_neighbor_vertices
                modRij = np.linalg.norm(z[:2,k-1,None,:] - z[:2,k-1,:,None], axis = 0)
                N_i = np.diff(indptr_neigh)
                segments = np.concatenate([ np.array([ np.broadcast_to(z[:2,k-1,i,None], (2, N_i[i])), (np.broadcast_to(z[:2,k-1,i,None], (2, N_i[i])) + z[:2,k-1, neighbours[indptr_neigh[i]:indptr_neigh[i+1]]])/2 ]).transpose((2,0,1)) /lWalls*100 for i in range(nParticles) ])

                if isSocialSwitching_lowerV:
                    modVel = np.linalg.norm(z[2:,k-1], axis = 0)
                    deltaModV = modVel[None,:] - modVel[:,None] 
                    colors = np.concatenate([ ['white' if deltaModV[i, j] <= 0 else 'tab:blue' if modRij[i][j] >= d_nn else 'tab:orange' for j in neighbours[indptr_neigh[i]:indptr_neigh[i+1]] ] for i in range(nParticles) ])
                elif isSocialSwitching_F_SocialOff:
                    colors = np.concatenate([ ['white' if ~i_SocialOn[i] else 'tab:blue' if modRij[i][j] >= d_nn else 'tab:orange' for j in neighbours[indptr_neigh[i]:indptr_neigh[i+1]] ] for i in range(nParticles) ])
                else:
                    colors = np.concatenate([ ['tab:blue' if modRij[i][j] >= d_nn else 'tab:orange' for j in neighbours[indptr_neigh[i]:indptr_neigh[i+1]] ] for i in range(nParticles) ])

                plt.gca().add_collection( LineCollection(segments, zorder = 0, colors = colors, lw = 0.7, alpha = 0.5) )

        if isSocialSwitching_lowerV or isSocialSwitching_F_SocialOff:
            plt.scatter(*posDraw[:,i_SocialOn], s=3 )
            plt.quiver(*posDraw[:,i_SocialOn], z[2,k-1,i_SocialOn], z[3,k-1,i_SocialOn], color = 'b', headaxislength = 0, headlength=0, width = widthVectors*2, scale = scaleVectors_v, scale_units = 'xy', angles='xy', label = r'$\vec{v}$ social', alpha = alphaVectors)
            plt.scatter(*posDraw[:,~i_SocialOn], s=3, color = color_antisocial )
            plt.quiver(*posDraw[:,~i_SocialOn], z[2,k-1,~i_SocialOn], z[3,k-1,~i_SocialOn], color = color_antisocial, headaxislength = 0, headlength=0, width = widthVectors*2, scale = scaleVectors_v, scale_units = 'xy', angles='xy', label = r'$\vec{v}$ non-social', alpha = alphaVectors)
        else:    
            plt.scatter(*posDraw, s=3 )
            plt.quiver(*posDraw, z[2,k-1,:], z[3,k-1,:], color = 'b', headaxislength = 0, headlength=0, width = widthVectors*2, scale = scaleVectors_v, scale_units = 'xy', angles='xy', label = r'$\vec{v}$', alpha = alphaVectors)

        if isSocialSwitching_F_SocialOff: plt.quiver(*posDraw, *F_SocialOff[:,k-1], color = 'green', headaxislength = 0, headlength=0, width = widthVectors, scale = scaleVectors_F, scale_units = 'xy', angles='xy', label = 'F Social Off', alpha = alphaVectors)

        plt.quiver(*posDraw, *F_Soc_alig[:,k-1], color = 'magenta', headaxislength = 0, headlength=0, width = widthVectors, scale = scaleVectors_F, scale_units = 'xy', angles='xy', label = r'$\vec{F}^\textrm{alig}$', alpha = alphaVectors)

        plt.quiver(*posDraw, *F_Soc_atracRep[:,k-1], color = 'red', headaxislength = 0, headlength=0, width = widthVectors, scale = scaleVectors_F, scale_units = 'xy', angles='xy', label = r'$\vec{F}^\textrm{att-rep}$', alpha = alphaVectors)

        plt.quiver(*posDraw, *F_fricProp[:,k-1], color = 'cyan', headaxislength = 0, headlength=0, width = widthVectors, scale = scaleVectors_F, scale_units = 'xy', angles='xy', label = r'$\vec{F}^\textrm{fric-prop}$', alpha = alphaVectors)

        posCM = np.mean(z[:2,k-1,:], axis = 1)
        plt.xlim(np.array([posCM[0]- lGroup, posCM[0]+ lGroup])/lWalls*100)
        plt.ylim(np.array([posCM[1]- lGroup, posCM[1]+ lGroup])/lWalls*100)

        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xlabel(r'$x~(\textrm{cm})$')
        plt.ylabel(r'$y~(\textrm{cm})$')
        plt.gca().set_aspect(1./plt.gca().get_data_ratio())

        plt.savefig('sim/pos_%d.png' %(tSave), bbox_inches='tight', dpi = 300)

        tSave += 1
        plt.close()

sol, F_Social_alig, F_Social_atracRep, F_fricProp, F_noise, F_SocialOff = z[:,1::int(1/dt)][:,tTransient:], F_Soc_alig[:,::int(1/dt)][:,tTransient:], F_Soc_atracRep[:,::int(1/dt)][:,tTransient:], F_fricProp[:,::int(1/dt)][:,tTransient:], F_noise[:,::int(1/dt)][:,tTransient:] / np.sqrt(dt), F_SocialOff[:,::int(1/dt)][:,tTransient:]


sol = sol.transpose((1,2,0)) 
F_Social_alig = F_Social_alig.transpose((1,2,0))
F_fricProp = F_fricProp.transpose((1,2,0))
F_Social_atracRep = F_Social_atracRep.transpose((1,2,0))
F_noise = F_noise.transpose((1,2,0))
F_SocialOff = F_SocialOff.transpose((1,2,0))
F_total = F_Social_alig + F_fricProp + F_Social_atracRep + F_SocialOff + F_noise  


sigma = 2
truncate = 10
pos = gaussian_filter1d(sol[...,:2], sigma = sigma, truncate = truncate, axis = 0)
vel = pos[1:] - pos[:-1]
acc = vel[1:] - vel[:-1]
pos = pos[1:-1]
vel = vel[:-1]
F_Social_alig = F_Social_alig[1:-1]
F_fricProp = F_fricProp[1:-1]
F_Social_atracRep = F_Social_atracRep[1:-1]
F_noise = F_noise[1:-1]
F_total = F_total[1:-1]
F_SocialOff = F_SocialOff[1:-1]

F_total = gaussian_filter1d(F_total, sigma = sigma, truncate = truncate, axis = 0)

pos = pos / lWalls * 100
vel = vel / lWalls * 100 * fps
acc = acc / lWalls * 100 * fps**2

F_Social_alig, F_Social_atracRep, F_fricProp, F_noise, F_SocialOff = F_Social_alig/ lWalls * 100 * fps**2, F_Social_atracRep/ lWalls * 100 * fps**2, F_fricProp/ lWalls * 100 * fps**2, F_noise/ lWalls * 100 * fps**2, F_SocialOff/ lWalls * 100 * fps**2


#Calculations
nTime = pos.shape[0]

thetaI = np.arctan2(-vel[..., 0], vel[..., 1]) 

rij = np.linalg.norm(pos[...,:,None,:] - pos[...,None,:,:],axis=-1)
nn_i = np.argpartition(rij, 1, axis = -1)[...,1] #Nearest neighbour

#Relative velocity of the individual with the nearest neighbour. Y coordinate is along the direction of motion of i.
velNn_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), vel - np.take_along_axis(vel, nn_i[...,None], axis = 1))

#Relative position of the individual with the nearest neighbour. Y coordinate is along the direction of motion of i.
posNn_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), pos - np.take_along_axis(pos, nn_i[...,None], axis = 1))

#Acceleration of the individual. Y coordinate is along the direction of motion of i.
acc_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), acc)

Output()

## Attraction-repulsion plots

In [9]:
folder = 'atracRep_Nn' #Need to create this folder!!

xLabel = r'$x_i - x_{NN}~(\textrm{cm})$'
yLabel = r'$y_i - y_{NN}~(\textrm{cm})$'
L = 300/lWalls*100
x_r = np.linspace(-L, L, 200)
y_r = np.linspace(-L, L, 200)
Bins_arrows = np.linspace(-L, L, 20)
modAcc_lim = 0.2/lWalls*100*fps**2

plotCounts2D(posNn_i[...,0].flatten(), posNn_i[...,1].flatten(), x_r, y_r, xLabel, yLabel, '', '', folder, isLog_z=True, tagName = tag, output='pdf')

plotAverage2D_VecField(posNn_i[...,0].flatten(), posNn_i[...,1].flatten(), acc_i[...,0].flatten(), acc_i[...,1].flatten(), x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{a}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn', zLim = [0, modAcc_lim], tagName =tag, output='pdf')

F_fricProp_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_fricProp)
plotAverage2D_VecField(posNn_i[...,0].flatten(), posNn_i[...,1].flatten(), F_fricProp_i[...,0].flatten(), F_fricProp_i[...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{F}^\textrm{fric-prop}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn', zLim = [0, modAcc_lim], tagName =tag, output='pdf')

F_Social_atracRep_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_Social_atracRep)
plotAverage2D_VecField(posNn_i[...,0].flatten(), posNn_i[...,1].flatten(), F_Social_atracRep_i[...,0].flatten(), F_Social_atracRep_i[...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{F}^\textrm{att-rep}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn', zLim = [0, modAcc_lim], tagName =tag, output='pdf')


F_Social_alig_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_Social_alig)
plotAverage2D_VecField(posNn_i[...,0].flatten(), posNn_i[...,1].flatten(), F_Social_alig_i[...,0].flatten(), F_Social_alig_i[...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{F}^\textrm{alig}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn',  zLim = [0, modAcc_lim], tagName =tag, output='pdf')


if isSocialSwitching_F_SocialOff:
    F_SocialOff_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_SocialOff)
    cond = F_SocialOff_i.any(axis = -1)
    plotCounts2D(posNn_i[cond][...,0].flatten(), posNn_i[cond][...,1].flatten(), x_r, y_r, xLabel, yLabel, tag + ' F_SocialOff', folder, isLog_z=True)
    plotAverage2D_VecField(posNn_i[cond][...,0].flatten(), posNn_i[cond][...,1].flatten(), F_SocialOff_i[cond][...,0].flatten(), F_SocialOff_i[cond][...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel =  r'$\vec{F}^\textrm{rand}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = tag, folder=folder, cmap = 'YlGn',  zLim = [0, modAcc_lim], output='pdf')

## Alignment plots

In [7]:
folder = 'alig_Nn' #Need to create this folder!!

xLabel = r'$v_i^x - v_{NN}^x~(\textrm{cm}/\textrm{s})$'
yLabel = r'$v_i^y - v_{NN}^y~(\textrm{cm}/\textrm{s})$'
vMax = 15/lWalls*100*fps
x_r = np.linspace(-vMax, vMax, 200)
y_r = np.linspace(-vMax, vMax, 200)
Bins_arrows = np.linspace(-vMax, vMax, 20)
modAcc_lim = 0.2/lWalls*100*fps**2

plotCounts2D(velNn_i[...,0].flatten(), velNn_i[...,1].flatten(), x_r, y_r, xLabel, yLabel, '', '', folder, isLog_z=True, tagName = tag, output='pdf')

plotAverage2D_VecField(velNn_i[...,0].flatten(), velNn_i[...,1].flatten(), acc_i[...,0].flatten(), acc_i[...,1].flatten(), x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{a}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn', zLim = [0, modAcc_lim], tagName =tag, output='pdf')

F_fricProp_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_fricProp)
plotAverage2D_VecField(velNn_i[...,0].flatten(), velNn_i[...,1].flatten(), F_fricProp_i[...,0].flatten(), F_fricProp_i[...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{F}^\textrm{fric-prop}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn', zLim = [0, modAcc_lim], tagName =tag, output='pdf')

F_Social_atracRep_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_Social_atracRep)
plotAverage2D_VecField(velNn_i[...,0].flatten(), velNn_i[...,1].flatten(), F_Social_atracRep_i[...,0].flatten(), F_Social_atracRep_i[...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{F}^\textrm{att-rep}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn', zLim = [0, modAcc_lim], tagName =tag, output='pdf')


F_Social_alig_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_Social_alig)
plotAverage2D_VecField(velNn_i[...,0].flatten(), velNn_i[...,1].flatten(), F_Social_alig_i[...,0].flatten(), F_Social_alig_i[...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel = r'$\vec{F}^\textrm{alig}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = '', folder=folder, cmap = 'YlGn',  zLim = [0, modAcc_lim], tagName =tag, output='pdf')


if isSocialSwitching_F_SocialOff:
    F_SocialOff_i = np.einsum('...mn, ...n -> ...m', R_vec(-thetaI), F_SocialOff)
    cond = F_SocialOff_i.any(axis = -1)
    plotCounts2D(velNn_i[cond][...,0].flatten(), velNn_i[cond][...,1].flatten(), x_r, y_r, xLabel, yLabel, tag + ' F_SocialOff', folder, isLog_z=True)
    plotAverage2D_VecField(velNn_i[cond][...,0].flatten(), velNn_i[cond][...,1].flatten(), F_SocialOff_i[cond][...,0].flatten(), F_SocialOff_i[cond][...,1].flatten(),  x_r, y_r, Bins_arrows, Bins_arrows, (Bins_arrows[:-1] + Bins_arrows[1:])/2, (Bins_arrows[:-1] + Bins_arrows[1:])/2, arrowCutoffPerc = 0.05, xLabel = xLabel, yLabel = yLabel, zLabel =  r'$\vec{F}^\textrm{rand}_i~(\textrm{cm}/\textrm{s}^2)$', zPerc = 5, title = tag, folder=folder, cmap = 'YlGn',  zLim = [0, modAcc_lim], output='pdf')